### Food Image Classification With a CNN w/ Transfer Learning via VGG16

In [1]:
# load the necessary packages
import tensorflow as tf
from tensorflow.keras.layers import Input, AveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.models import Model
import pandas as pd


import os
import numpy as np
import matplotlib.pyplot as plt

### Load and Preprocess Data for Model

In [2]:
# setup model architechture
# remove dense top layers from pre-trained model
base_model = VGG16(weights='imagenet', 
              include_top=False, 
              input_tensor=(Input(shape=(256,256,3)))
             )

#freeze training in pre-trained layers
for layer in base_model.layers:
    layer.trainable = False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [3]:
#add in final layers of model
head_model = base_model.output
head_model = AveragePooling2D(pool_size=(8,8))(head_model)
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(256, activation='relu')(head_model)
head_model = Dropout(0.4)(head_model)
head_model = Dense(101, activation='softmax')(head_model)

#place new head model on top of pre-trained network
model = Model(inputs=base_model.input, outputs=head_model)

In [4]:
# compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
csv_logger = CSVLogger('../history/VGG16_v3.log')

# initialize early stopping callback
es_callback = EarlyStopping(monitor="val_loss", patience=10)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [7]:
batch_size = 25
epochs = 1000
IMG_HEIGHT = 256
IMG_WIDTH = 256
train_dir = '../data/resized/train'
valid_dir = '../data/resized/valid'
dropout_rate = 0.4

In [8]:
train_image_generator = ImageDataGenerator(rescale=1./255)
valid_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
# initialize training image iterator
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                          directory=train_dir,
                                                          shuffle=True,
                                                          target_size=(IMG_HEIGHT,IMG_WIDTH))

Found 75750 images belonging to 101 classes.


In [ ]:
# initialize test image iterator
valid_data_gen = valid_image_generator.flow_from_directory(batch_size=batch_size,
                                                          directory=valid_dir,
                                                          shuffle=True,
                                                          target_size=(IMG_HEIGHT,IMG_WIDTH))

Found 12625 images belonging to 101 classes.


In [ ]:
# train model
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=75750//batch_size,
    epochs=epochs,
    validation_data=valid_data_gen,
    validation_steps=12625//batch_size,
    workers=1,
    use_multiprocessing=False,
    callbacks=[es_callback, csv_logger]
)

Epoch 1/1000
3030/3030 [==============================] - 1963s 648ms/step - loss: 3.9126 - accuracy: 0.1079 - val_loss: 3.2308 - val_accuracy: 0.2419
Epoch 2/1000
3030/3030 [==============================] - 1957s 646ms/step - loss: 3.3899 - accuracy: 0.1968 - val_loss: 2.9355 - val_accuracy: 0.3025
Epoch 3/1000
3030/3030 [==============================] - 1944s 642ms/step - loss: 3.2374 - accuracy: 0.2262 - val_loss: 2.8329 - val_accuracy: 0.3213
Epoch 4/1000
3030/3030 [==============================] - 1957s 646ms/step - loss: 3.1525 - accuracy: 0.2440 - val_loss: 2.7404 - val_accuracy: 0.3364
Epoch 5/1000
3030/3030 [==============================] - 1962s 648ms/step - loss: 3.0990 - accuracy: 0.2531 - val_loss: 2.6443 - val_accuracy: 0.3491
Epoch 6/1000
3030/3030 [==============================] - 1953s 645ms/step - loss: 3.0483 - accuracy: 0.2638 - val_loss: 2.6278 - val_accuracy: 0.3587
Epoch 7/1000
3030/3030 [==============================] - 1945s 642ms/step - loss: 3.0225 - ac

In [ ]:
# if there is not a saved model, save model for future use
if not os.path.isdir('../saved_models'):
    os.makedirs('../saved_models')
model.save('../saved_models/CNN_VGG16_MODEL_V3.h5')

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()